In [ ]:
from importlib import reload
import os
import numpy as np
from scipy.signal import medfilt, medfilt2d, get_window
from scipy.ndimage import median_filter

from obspy.clients.filesystem.sds import Client
from obspy.clients.fdsn import RoutingClient
from obspy.core import UTCDateTime as UTC, read
from obspy.signal import util

import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

from data_quality_control import sds_db, base

In [ ]:
median_filter()

In [ ]:
medfilt2d()

## Define parameters

In [ ]:
# NSLC
nslc_code = "GR.BFO..BHZ"

overlap = 60 #3600
fmin, fmax = (4, 14)
nperseg = 2048
winlen_in_s = 3600
proclen = 24*3600

outdir = 'output'

sds_root = os.path.abspath('../../sample_sds/')
inventory_routing_type = "eida-routing"

In [ ]:
from data_quality_control import analysis #, util

In [ ]:
# Only for display in documentation!
from IPython.core.display import display, HTML 

First, we initialize an Analyzer by setting the path to the 
HDF5-data (`outdir`), a station code and the `fileunit`, i.e.
which name format the HDF5-files have, that we want to analyze.

The initial object does not have any data yet.

In [ ]:
reload(analysis)
#reload(util)
lyza = analysis.Analyzer(outdir, nslc_code,
                            fileunit="year")

In [ ]:
print(lyza)

We can inquire which files and time ranges are available for
the given code, location and fileunit.

In [ ]:
files = lyza.get_available_datafiles()
print(files)

In [ ]:
lyza.get_available_timerange()

## View data for time range

In order to view the data, the amplitudes and spectra are treated differently.
Amplitudes are loaded as they are in the HDF5-file. Thus, we obtain an array of
shape `N_proclen x N_winlen`. The sample data covers 16 days and we used 
`proclen_seconds = 86400`, i.e. 1 day, so the first dimension is 16. 
The `winlen_seconds = 3600`, thus 24 windows per day which gives the second dimension
of the amplitude array.

For the spectra, there are two options:
1. load all spectra within a specific time range
2. load spectra for selected times given as list

The spectra are stored in the HDF5-files as 3D arrays. The
first two dimensions correspond to those of the amplitude
array; the third dimension is the frequency axis.
In contrast, the Analyzer flattens the first to dimensions,
i.e. the resulting array is basically a spectrogram, thus
a sequence of spectra over time.

This allows to select spectra only for specific times.
For example, one may want to select only those hours
where the wind speed is in a specific range.

In [ ]:
startdate = UTC("2020-12-25")
enddate = UTC("2021-01-15")

In [ ]:
DATA = lyza.get_data(startdate, enddate)

In [ ]:
lyza

In [ ]:
DATA

In [ ]:
DATA.amplitudes.shape

In [ ]:
DATA.psds.shape

In [ ]:
def get_overlapping_frames(x, inc, winlen_samples):
    
    nf = (x.size-winlen_samples) // inc +1
    #print(nf)
    
    indf = inc * np.arange(nf)
    #print(indf)
    #print(np.expand_dims(indf, 1) + 
    #      np.expand_dims(np.arange(winlen_samples), 0))
    f = x[np.expand_dims(indf, 1) + 
          np.expand_dims(np.arange(winlen_samples), 0)]

    return f

In [ ]:
psds = DATA.psds.reshape((-1, DATA.psds.shape[-1])).copy()
psds.shape

In [ ]:
x = psds[:24,0]
x.shape

In [ ]:
f = get_overlapping_frames(x, 2, 6)
f.shape

In [ ]:
smoothed = []
for psd in psds.T:
    #print(psd.shape)
    f = get_overlapping_frames(psd, 3, 6)
    x = np.nanmedian(f, axis=1)
    smoothed.append(x)
smoothed = np.array(smoothed).T

In [ ]:
smoothed.shape

In [ ]:
inc = 3
winlen = 6
smoothed = np.array([np.nanmedian(get_overlapping_frames(x, inc, winlen),axis=1) for x in psds.T]).T

smoothed.shape

In [ ]:
fm = np.nanmedian(f, axis=1)
fm

# Figuring out how my get_overlapping_frames() work

In [ ]:
sdsclient = Client(sds_root)

In [ ]:
endtime = startdate+24*3600
st = sdsclient.get_waveforms(*nslc_code.split("."), startdate, endtime)
st = st.trim(startdate-60, endtime+60, pad=True, fill_value=0)
tr = st[0]
x = np.arange(1,24+1).repeat(72000)
print(x.shape)
print(tr.stats.npts)
tr.data[60*20+1:-60*20] = x

In [ ]:
tr.data.size % 24 # 74400

In [ ]:
tr.plot(endtime=startdate+600);

In [ ]:
procparams = base.ProcessingParameters()

In [ ]:
procparams

In [ ]:
nf = int(procparams.proclen_seconds/
        procparams.winlen_seconds)

In [ ]:
nf

In [ ]:
f, taps = get_overlapping_tapered_frames(tr, startdate, 24, int(3600*20), 60*20)

In [ ]:
f.shape

In [ ]:
plt.imshow(f, aspect="auto")

In [ ]:
#plt.plot(f[10,60*20+1:-60*20])
plt.plot(f[11,:])
#plt.xlim(-1, 10)

In [ ]:
3600*20

In [ ]:
6*2*100*2

In [ ]:
x = np.arange(24).repeat(74400)

In [ ]:
x

In [ ]:
def get_overlapping_tapered_frames(tr, starttime, nf, winlen_samples,
                           taper_samples):
    sr = tr.stats.sampling_rate
    
    # Samples in window including tapers
    nwin = int(winlen_samples + 2*taper_samples)
    
    # Total number of samples of trace to process
    proclen_samples = int(nf * winlen_samples + 2*taper_samples)
    
    # Cut out the needed data
    x = tr.slice(starttime-taper_samples/sr).data[:proclen_samples]
    
    # Ratio of tapers to total window size
    a =  2*taper_samples / nwin
    win = get_window(('tukey', a), nwin, fftbins=False)
    
    # From obspy.signal.enframe()
    #nx = len(x)
    #nwin = len(win)
    if (len(win) == 1):
        length = win
    else:
        length = nwin
    #nf = int(np.fix((nx - length + winlen_samples) // winlen_samples))
    # f = np.zeros((nf, length))
    indf = winlen_samples * np.arange(nf)
    f = x[np.expand_dims(indf, 1) + 
          np.expand_dims(np.arange(length), 0)]
    print(indf)
    print(length)
    print(np.expand_dims(indf, 1) + 
          np.expand_dims(np.arange(length), 0))
    #f = f * win
    #f[np.any(np.isnan(f), axis=1),:] = np.nan
    #no_win, _ = f.shape
    return f, taper_samples

In [ ]:
endtime = startdate+24*3600
st = sdsclient.get_waveforms(*nslc_code.split("."), startdate, endtime)
#st = st.trim(startdate-60, endtime+60, pad=True, fill_value=0)
tr = st[0]

sr = 0.01
x = np.arange(1,24+1).repeat(int(3600*sr))
#x = np.insert()
print(x.shape)
print(x.size/24)

tr.data = x
tr.stats.sampling_rate = sr
tr.stats.starttime = startdate
tr = tr.trim(startdate-600, endtime+600, pad=True, fill_value=0)
print(tr.stats.npts)

In [ ]:
tr.stats.starttime, tr.stats.endtime

In [ ]:
tr.plot();

In [ ]:
f, taps = get_overlapping_tapered_frames(tr, startdate, 24, int(3600*sr), 600*sr)

In [ ]:
f.shape

In [ ]:
plt.imshow(f, aspect="auto")

In [ ]:
f[1,:]

In [ ]:
f[0, 6:6+36]

In [ ]:
x2 = np.append(x, np.zeros(10))

In [ ]:
f1 = get_overlapping_frames(x2, 18, 36 )

In [ ]:
f1.shape

In [ ]:
plt.imshow(f1, aspect="auto")

In [ ]:
plt.plot(f1[-1,:])